In [ ]:
#NAME : OJAS SINGH             REG NO : 21BDS0187

In [9]:
from math import log
import operator
import csv

def load_dataset(filename):
    with open(filename, 'r') as file:
        csv_reader = csv.reader(file)
        headers = next(csv_reader)
        dataset = [row for row in csv_reader]
    return dataset, headers[:-1]

def entropy(data):
    entries = len(data)
    labels = {}
    for feat in data:
        label = feat[-1]
        labels[label] = labels.get(label, 0) + 1
    entropy_val = 0.0
    for key in labels:
        prob = float(labels[key])/entries
        entropy_val -= prob * log(prob, 2)
    return entropy_val

def split_data(data, axis, value):
    return [feat[:axis] + feat[axis+1:] for feat in data if feat[axis] == value]

def choose_best_feature(data):
    num_features = len(data[0]) - 1
    base_entropy = entropy(data)
    best_info_gain = 0.0
    best_feature = -1

    for i in range(num_features):
        feat_values = {ex[i] for ex in data}
        new_entropy = 0.0
        for value in feat_values:
            subset = split_data(data, i, value)
            prob = len(subset)/float(len(data))
            new_entropy += prob * entropy(subset)
        info_gain = base_entropy - new_entropy
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = i
    return best_feature

def majority_class(class_list):
    class_count = {}
    for vote in class_list:
        class_count[vote] = class_count.get(vote, 0) + 1
    return max(class_count.items(), key=operator.itemgetter(1))[0]

def build_tree(data, labels):
    class_list = [ex[-1] for ex in data]
    
    if class_list.count(class_list[0]) == len(class_list):
        return class_list[0]
    if len(data[0]) == 1:
        return majority_class(class_list)
    
    best_feat = choose_best_feature(data)
    best_feat_label = labels[best_feat]
    
    tree = {best_feat_label: {}}
    del(labels[best_feat])
    
    feat_values = {ex[best_feat] for ex in data}
    for value in feat_values:
        sub_labels = labels[:]
        subset = split_data(data, best_feat, value)
        tree[best_feat_label][value] = build_tree(subset, sub_labels)
    
    return tree

# Usage example with your dataset
dataset, labels = load_dataset('tennis.csv')
decision_tree = build_tree(dataset, labels)
print(decision_tree)


{'Outlook': {'Sunny': {'Humidity': {'Normal': 'Yes', 'High': 'No'}}, 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}, 'Overcast': 'Yes'}}
